In [2]:
import pandas as pd
import os

# Benchmarks

In [ ]:
path_to_results = os.getcwd() + "/results/"


index_pd = pd.MultiIndex.from_tuples(
    [
        ("RUMBoost", "Constant"),
        ("RUMBoost", "Linear"),
        # ("RUMBoost", "Cubic"),
        ("RUMBoost", "Constant + linear"),
        # ("RUMBoost", "Constant + cubic"),
        # ("APLR", "Linear"),
    ]
)

columns_pd = pd.MultiIndex.from_product(
    [("LPMC", "SwissMetro"), ("Monotonic", "Non-monotonic")]
)

benchmark_table_test = pd.DataFrame(
    {},
    columns=columns_pd,
    index=index_pd,
)
benchmark_table_train = pd.DataFrame(
    {},
    columns=columns_pd,
    index=index_pd,
)
benchmark_table_ct = pd.DataFrame(
    {},
    columns=columns_pd,
    index=index_pd,
)

for model in ["RUMBoost"]:  # , "APLR"]:
    for monotone in ["True", "False"]:
        for model_type in [
            "constant",
            "linear",
            "constant_linear",
        ]:  # ,"linear", "cubic","constant_cubic"]:
            for dataset in ["LPMC", "SwissMetro"]:
                if model == "APLR" and model_type != "linear":
                    continue
                try:
                    results = pd.read_csv(
                        f"{path_to_results}{dataset}/{model}/{model_type}/mono{monotone}/results_True.csv"
                    )
                except FileNotFoundError:
                    print(
                        f"Results file not found for {dataset}, {model}, {model_type}, monotone={monotone}"
                    )
                    continue
                monotone_c = "Monotonic" if monotone == "True" else "Non-monotonic"
                model_type_c = (
                    "Constant"
                    if model_type == "constant"
                    else (
                        "Linear"
                        if model_type == "linear"
                        else (
                            "Cubic"
                            if model_type == "cubic"
                            else (
                                "Constant + linear"
                                if model_type == "constant_linear"
                                else "Constant + cubic"
                            )
                        )
                    )
                )
                benchmark_table_test.loc[
                    (model, model_type_c), (dataset, monotone_c)
                ] = (results["loss_test"].values[0].round(4))
                benchmark_table_train.loc[
                    (model, model_type_c), (dataset, monotone_c)
                ] = (results["train_loss"].values[0].round(4))
                benchmark_table_ct.loc[(model, model_type_c), (dataset, monotone_c)] = (
                    round(results["train_time"].values[0] / 6, 2)
                )  # divide by 6 because 5 folds + final training

# adding DCMs - value calculated in biogeme_models_run.ipynb for test
# from the results file for train and computational time -
# MNL on LPMC is directly taken from RUMBoost paper
benchmark_table_test.loc[("DCM", "MNL"), ("SwissMetro", "Monotonic")] = 0.7950
benchmark_table_test.loc[("DCM", "MNL"), ("SwissMetro", "Non-monotonic")] = "-"
benchmark_table_test.loc[("DCM", "MNL"), ("LPMC", "Monotonic")] = 0.7095
benchmark_table_test.loc[("DCM", "MNL"), ("LPMC", "Non-monotonic")] = "-"
benchmark_table_test.loc[("DCM", "PWL-MNL"), ("SwissMetro", "Monotonic")] = 0.7673
benchmark_table_test.loc[("DCM", "PWL-MNL"), ("SwissMetro", "Non-monotonic")] = "-"
benchmark_table_test.loc[("DCM", "PWL-MNL"), ("LPMC", "Monotonic")] = "-"
benchmark_table_test.loc[("DCM", "PWL-MNL"), ("LPMC", "Non-monotonic")] = "-"

benchmark_table_train.loc[("DCM", "MNL"), ("SwissMetro", "Monotonic")] = 0.7394
benchmark_table_train.loc[("DCM", "MNL"), ("SwissMetro", "Non-monotonic")] = "-"
benchmark_table_train.loc[("DCM", "MNL"), ("LPMC", "Monotonic")] = 0.6913
benchmark_table_train.loc[("DCM", "MNL"), ("LPMC", "Non-monotonic")] = "-"
benchmark_table_train.loc[("DCM", "PWL-MNL"), ("SwissMetro", "Monotonic")] = 0.6890
benchmark_table_train.loc[("DCM", "PWL-MNL"), ("SwissMetro", "Non-monotonic")] = "-"
benchmark_table_train.loc[("DCM", "PWL-MNL"), ("LPMC", "Monotonic")] = "-"
benchmark_table_train.loc[("DCM", "PWL-MNL"), ("LPMC", "Non-monotonic")] = "-"

benchmark_table_ct.loc[("DCM", "MNL"), ("SwissMetro", "Monotonic")] = 1.22
benchmark_table_ct.loc[("DCM", "MNL"), ("SwissMetro", "Non-monotonic")] = "-"
benchmark_table_ct.loc[("DCM", "MNL"), ("LPMC", "Monotonic")] = 242.14
benchmark_table_ct.loc[("DCM", "MNL"), ("LPMC", "Non-monotonic")] = "-"
benchmark_table_ct.loc[("DCM", "PWL-MNL"), ("SwissMetro", "Monotonic")] = 865.03
benchmark_table_ct.loc[("DCM", "PWL-MNL"), ("SwissMetro", "Non-monotonic")] = "-"
benchmark_table_ct.loc[("DCM", "PWL-MNL"), ("LPMC", "Monotonic")] = "-"
benchmark_table_ct.loc[("DCM", "PWL-MNL"), ("LPMC", "Non-monotonic")] = "-"

In [22]:
benchmark_table_test

LPMC               SwissMetro              
                           Monotonic Non-monotonic  Monotonic Non-monotonic
RUMBoost Constant             0.6741        0.6745     0.7675        0.7758
         Linear               0.6714        0.6706     0.7688        0.7702
         Constant + linear    0.6727        0.6724     0.7676        0.7716
DCM      MNL                  0.7095             -      0.795             -
         PWL-MNL                   -             -     0.7673             -

In [ ]:
benchmark_table_test.to_latex(
    f"{path_to_results}benchmark_table_test.tex",
)

In [24]:
benchmark_table_ct

LPMC               SwissMetro              
                           Monotonic Non-monotonic  Monotonic Non-monotonic
RUMBoost Constant              27.79         37.83       6.02         56.13
         Linear              1027.09       1998.15     163.22        467.04
         Constant + linear    217.78        301.47      48.71        293.16
DCM      MNL                  242.14             -       1.22             -
         PWL-MNL                   -             -     865.03             -

In [29]:
benchmark_table_ct.to_latex(
    f"{path_to_results}benchmark_table_ct.tex", decimal=".", float_format="%.2f"
)

In [26]:
benchmark_table_train

LPMC               SwissMetro              
                           Monotonic Non-monotonic  Monotonic Non-monotonic
RUMBoost Constant             0.6397        0.6359     0.6704        0.6373
         Linear               0.6462        0.6426     0.6883        0.6819
         Constant + linear     0.643        0.6394     0.6781        0.6549
DCM      MNL                  0.6913             -     0.7394             -
         PWL-MNL                   -             -      0.689             -

In [28]:
benchmark_table_train.to_latex(
    f"{path_to_results}benchmark_table_train.tex", decimal=".", float_format="%.4f"
)